# Criação do DataSet e Análise das Rotulações

## Pacotes

In [1]:
import pandas as pd
import jsonlines
import numpy as np
import re

from nltk.tokenize.api import TokenizerI
from nltk.tokenize.util import align_tokens
from nltk.tokenize.destructive import MacIntyreContractions

## Tokenizador

In [7]:
class TreebankWordTokenizer(TokenizerI):
    r"""
    The Treebank tokenizer uses regular expressions to tokenize text as in Penn Treebank.
    This is the method that is invoked by ``word_tokenize()``.  It assumes that the
    text has already been segmented into sentences, e.g. using ``sent_tokenize()``.

    This tokenizer performs the following steps:

    - split standard contractions, e.g. ``don't`` -> ``do n't`` and ``they'll`` -> ``they 'll``
    - treat most punctuation characters as separate tokens
    - split off commas and single quotes, when followed by whitespace
    - separate periods that appear at the end of line

        >>> from nltk.tokenize import TreebankWordTokenizer
        >>> s = '''Good muffins cost $3.88\nin New York.  Please buy me\ntwo of them.\nThanks.'''
        >>> TreebankWordTokenizer().tokenize(s)
        ['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York.', 'Please', 'buy', 'me', 'two', 'of', 'them.', 'Thanks', '.']
        >>> s = "They'll save and invest more."
        >>> TreebankWordTokenizer().tokenize(s)
        ['They', "'ll", 'save', 'and', 'invest', 'more', '.']
        >>> s = "hi, my name can't hello,"
        >>> TreebankWordTokenizer().tokenize(s)
        ['hi', ',', 'my', 'name', 'ca', "n't", 'hello', ',']
    """

    # starting quotes
    STARTING_QUOTES = [
        (re.compile(r"^\""), r"``"),
        (re.compile(r"(``)"), r" \1 "),
        (re.compile(r"([ \(\[{<])(\"|\'{2})"), r"\1 `` "),
    ]

    # punctuation
    PUNCTUATION = [
        (re.compile(r"([:,])([^\d])"), r" \1 \2"),
        (re.compile(r"([:,])$"), r" \1 "),
        (re.compile(r"\.\.\."), r" ... "),
        (re.compile(r"[;@#&-/]"), r" \g<0> "),
        (re.compile(r"(\d{2}) / (\d{2}) / (\d{2,4})"), r"\1/\2/\3"),
        (
            re.compile(r'([^\.])(\.)([\]\)}>"\']*)\s*$'),
            r"\1 \2\3 ",
        ),  # Handles the final period.
        (re.compile(r"[?!]"), r" \g<0> "),
        (re.compile(r"([^'])' "), r"\1 ' "),
    ]

    # Pads parentheses
    PARENS_BRACKETS = (re.compile(r"[\]\[\(\)\{\}\<\>]"), r" \g<0> ")

    # Optionally: Convert parentheses, brackets and converts them to PTB symbols.
    CONVERT_PARENTHESES = [
        (re.compile(r"\("), "-LRB-"),
        (re.compile(r"\)"), "-RRB-"),
        (re.compile(r"\["), "-LSB-"),
        (re.compile(r"\]"), "-RSB-"),
        (re.compile(r"\{"), "-LCB-"),
        (re.compile(r"\}"), "-RCB-"),
    ]

    DOUBLE_DASHES = (re.compile(r"--"), r" -- ")

    # ending quotes
    ENDING_QUOTES = [
        (re.compile(r'"'), " '' "),
        (re.compile(r"(\S)(\'\')"), r"\1 \2 "),
        (re.compile(r"([^' ])('[sS]|'[mM]|'[dD]|') "), r"\1 \2 "),
        (re.compile(r"([^' ])('ll|'LL|'re|'RE|'ve|'VE|n't|N'T) "), r"\1 \2 "),
    ]

    # List of contractions adapted from Robert MacIntyre's tokenizer.
    _contractions = MacIntyreContractions()
    CONTRACTIONS2 = list(map(re.compile, _contractions.CONTRACTIONS2))
    CONTRACTIONS3 = list(map(re.compile, _contractions.CONTRACTIONS3))

    def tokenize(self, text, convert_parentheses=False, return_str=False):
        for regexp, substitution in self.STARTING_QUOTES:
            text = regexp.sub(substitution, text)
        
        for regexp, substitution in self.PUNCTUATION:
            text = regexp.sub(substitution, text)

        # Handles parentheses.
        regexp, substitution = self.PARENS_BRACKETS
        text = regexp.sub(substitution, text)
        # Optionally convert parentheses
        if convert_parentheses:
            for regexp, substitution in self.CONVERT_PARENTHESES:
                text = regexp.sub(substitution, text)

        # Handles double dash.
        regexp, substitution = self.DOUBLE_DASHES
        text = regexp.sub(substitution, text)

        # add extra space to make things easier
        text = " " + text + " "

        for regexp, substitution in self.ENDING_QUOTES:
            text = regexp.sub(substitution, text)

        for regexp in self.CONTRACTIONS2:
            text = regexp.sub(r" \1 \2 ", text)
        for regexp in self.CONTRACTIONS3:
            text = regexp.sub(r" \1 \2 ", text)

        # We are not using CONTRACTIONS4 since
        # they are also commented out in the SED scripts
        # for regexp in self._contractions.CONTRACTIONS4:
        #     text = regexp.sub(r' \1 \2 \3 ', text)

        return text if return_str else text.split()

    def span_tokenize(self, text):
        r"""
        Uses the post-hoc nltk.tokens.align_tokens to return the offset spans.

            >>> from nltk.tokenize import TreebankWordTokenizer
            >>> s = '''Good muffins cost $3.88\nin New (York).  Please (buy) me\ntwo of them.\n(Thanks).'''
            >>> expected = [(0, 4), (5, 12), (13, 17), (18, 19), (19, 23),
            ... (24, 26), (27, 30), (31, 32), (32, 36), (36, 37), (37, 38),
            ... (40, 46), (47, 48), (48, 51), (51, 52), (53, 55), (56, 59),
            ... (60, 62), (63, 68), (69, 70), (70, 76), (76, 77), (77, 78)]
            >>> list(TreebankWordTokenizer().span_tokenize(s)) == expected
            True
            >>> expected = ['Good', 'muffins', 'cost', '$', '3.88', 'in',
            ... 'New', '(', 'York', ')', '.', 'Please', '(', 'buy', ')',
            ... 'me', 'two', 'of', 'them.', '(', 'Thanks', ')', '.']
            >>> [s[start:end] for start, end in TreebankWordTokenizer().span_tokenize(s)] == expected
            True

            Additional example
            >>> from nltk.tokenize import TreebankWordTokenizer
            >>> s = '''I said, "I'd like to buy some ''good muffins" which cost $3.88\n each in New (York)."'''
            >>> expected = [(0, 1), (2, 6), (6, 7), (8, 9), (9, 10), (10, 12),
            ... (13, 17), (18, 20), (21, 24), (25, 29), (30, 32), (32, 36),
            ... (37, 44), (44, 45), (46, 51), (52, 56), (57, 58), (58, 62),
            ... (64, 68), (69, 71), (72, 75), (76, 77), (77, 81), (81, 82),
            ... (82, 83), (83, 84)]
            >>> list(TreebankWordTokenizer().span_tokenize(s)) == expected
            True
            >>> expected = ['I', 'said', ',', '"', 'I', "'d", 'like', 'to',
            ... 'buy', 'some', "''", "good", 'muffins', '"', 'which', 'cost',
            ... '$', '3.88', 'each', 'in', 'New', '(', 'York', ')', '.', '"']
            >>> [s[start:end] for start, end in TreebankWordTokenizer().span_tokenize(s)] == expected
            True

        """
        raw_tokens = self.tokenize(text)

        # Convert converted quotes back to original double quotes
        # Do this only if original text contains double quote(s) or double
        # single-quotes (because '' might be transformed to `` if it is
        # treated as starting quotes).
        if ('"' in text) or ("''" in text):
            # Find double quotes and converted quotes
            matched = [m.group() for m in re.finditer(r"``|'{2}|\"", text)]

            # Replace converted quotes back to double quotes
            tokens = [
                matched.pop(0) if tok in ['"', "``", "''"] else tok
                for tok in raw_tokens
            ]
        else:
            tokens = raw_tokens

        for tok in align_tokens(tokens, text):
            yield tok


## Funções Auxiliares

 Função lê jsonl para lista

In [3]:
def jsonl2list(fileName):
    lista = []
    with jsonlines.open(fileName) as jsonl_file:
        for line in jsonl_file.iter():
            lista.append(line)
    return lista

Junta dados das duas saídas do Doccano

In [4]:
def merge(lista_A, lista_B):
    lista_merge = lista_A + lista_B
    return sorted(lista_merge, key=lambda k: k['id'])

Tokeniza os textos

In [5]:
## Metodo para tokenizar tudo (Todas as sentença do aruivo original .txt) 
## OBS: Gera como retorno duas listas

def tokenizeAn(lista):
    resultado = []
    referencia = []
    for index, item in enumerate(lista):
        sentenca = item['data']
        labels = item['label']
        wordSpansList = list(TreebankWordTokenizer().span_tokenize(sentenca))
        for wordSpan in wordSpansList:
            ## Adicionando a setença original, e o index da setença do arquivo original .txt
            referencia.append([sentenca, index,[wordSpan[0], wordSpan[1]]])
            newLine = [sentenca[wordSpan[0]:wordSpan[1]], 'o']
            for label in labels:
                if wordSpan[0] >= label[0] and wordSpan[1] <= label[1]:
                    newLine[1] = label[2]
                    break
            
            resultado.append(newLine)
    
    ## Metodo retorna resultado (sentanças tokenizada, juntamente com a rotulação da palavra)
    ## referencia é a setença completa junto com o index
    return resultado, referencia

Gera dataset no formato BIO

In [6]:
def makingTheDataset(lista):
    # print(lista_B)
    resultado = []
    contadorSentenca = 0
    for item in lista:
        contadorSentenca = contadorSentenca + 1
        sentenca = item['data']
        labels = item['label']
        wordSpansList = list(TreebankWordTokenizer().span_tokenize(sentenca))
        for wordSpan in wordSpansList:
            newLine = [sentenca[wordSpan[0]:wordSpan[1]], 'O', contadorSentenca]
            for label in labels:
                if wordSpan[0] == label[0] and wordSpan[1] <= label[1]:
                    newLine[1] = f'B-{label[2]}'
                    break
                elif wordSpan[0] > label[0] and wordSpan[1] <= label[1]:
                    newLine[1] = f'I-{label[2]}'
                    break
            
            resultado.append(newLine)
            
        
    return resultado


 Cohen Kappa

In [7]:
def cohen_kappa(ann1, ann2):
    """Computes Cohen kappa for pair-wise annotators.
    :param ann1: annotations provided by first annotator
    :type ann1: list
    :param ann2: annotations provided by second annotator
    :type ann2: list
    :rtype: float
    :return: Cohen kappa statistic
    """
    count = 0
    for an1, an2 in zip(ann1, ann2):
        if an1 == an2:
            count += 1
    A = count / len(ann1)  # observed agreement A (Po)

    uniq = set(ann1 + ann2)
    E = 0  # expected agreement E (Pe)
    for item in uniq:
        cnt1 = ann1.count(item)
        cnt2 = ann2.count(item)
        count = ((cnt1 / len(ann1)) * (cnt2 / len(ann2)))
        E += count

    return round((A - E) / (1 - E), 4)

Fleiss Kappa

In [8]:
def fleiss_kappa(M):
    """Computes Fleiss' kappa for group of annotators.
    :param M: a matrix of shape (:attr:'N', :attr:'k') with 'N' = number of subjects and 'k' = the number of categories.
        'M[i, j]' represent the number of raters who assigned the 'i'th subject to the 'j'th category.
    :type: numpy matrix
    :rtype: float
    :return: Fleiss' kappa score
    """
    N, k = M.shape  # N is # of items, k is # of categories
    n_annotators = float(np.sum(M[0, :]))  # # of annotators
    tot_annotations = N * n_annotators  # the total # of annotations
    category_sum = np.sum(M, axis=0)  # the sum of each category over all items
    # chance agreement
    p = category_sum / tot_annotations  # the distribution of each category over all annotations
    PbarE = np.sum(p * p)  # average chance agreement over all categories

    # observed agreement
    P = (np.sum(M * M, axis=1) - n_annotators) / (n_annotators * (n_annotators - 1))
    Pbar = np.sum(P) / N  # add all observed agreement chances per item and divide by amount of items

    return round((Pbar - PbarE) / (1 - PbarE), 4)

Função para geração de CSV

In [10]:
def gerarCSV(nome, df):
    df.to_csv(nome +".csv", encoding="utf-8")

## Lendo e juntando rotulações de todos rotuladores 

In [7]:
## Código para fazer leitura dos arquivos rotulado e não rotulados, realizar o merge entre eles
## e tokenizar as informações. Apenas Thiago e Arthur

# Dados Arthur
lista_A = jsonl2list("arthur_unknown.jsonl")
lista_B = jsonl2list("arthur.jsonl")

# Junta dados rotulados e não rotulados
lista_ArthurCompleta = merge(lista_A, lista_B)
dadosArthurFranco, referencia = tokenizeAn(lista_ArthurCompleta)

# Dados Thiago
lista_A = jsonl2list("thiago_unknown.jsonl")
lista_B = jsonl2list("thiago.jsonl")

# Junta dados rotulados e não rotulados
lista_ThiagoCompleta = merge(lista_A, lista_B)
dadosThiagoSalles = tokenizeAn(lista_ThiagoCompleta)[0]

Leitura e merge dos dados da priscilla

In [8]:
# Dados Priscilla
lista_A = jsonl2list("priscilla_unknown.jsonl")
lista_B = jsonl2list("priscilla.jsonl")

lista_PriscillaCompleta = []

## Código de força bruta para ordenar os dados da Priscilla, utiliza a lista já ordenada do 
## Arthur como referência para a ordenação, verificando a cada interação com toda a lista de
## dados rotulados ou não rotulados da Priscilla. (Devido a lista da Priscilla não possuir id sequencial)

for dado in lista_ArthurCompleta:
    vigia = True
    index_A = 0
    index_B = 0
    while vigia:
        if index_A < len(lista_A) and (dado['data'] == lista_A[index_A]['data']):
                lista_PriscillaCompleta.append(lista_A.pop(index_A))
                vigia = False
        elif index_B < len(lista_B) and dado['data'] == lista_B[index_B]['data']:
                lista_PriscillaCompleta.append(lista_B.pop(index_B))
                vigia = False
        if index_A >= len(lista_A) and index_B >= len(lista_B):
            break
        index_A += 1
        index_B += 1

## Tokenizar os dados da Priscilla

dadosPriscillSilva = tokenizeAn(lista_PriscillaCompleta)[0]

## Criando DataSet BIO

In [13]:
dsArthur = makingTheDataset(lista_ArthurCompleta)
dsThiago = makingTheDataset(lista_ThiagoCompleta)
dsPriscilla = makingTheDataset(lista_PriscillaCompleta)

Gerando csv com o dataset de cada rotulador

In [14]:
df = pd.DataFrame(dsArthur, columns = ['Palavras', 'Rotulo', 'Sentenca'])
gerarCSV("dsArthur", df)
df = pd.DataFrame(dsThiago, columns = ['Palavras', 'Rotulo', 'Sentenca'])
gerarCSV("dsThiago", df)
df = pd.DataFrame(dsPriscilla, columns = ['Palavras', 'Rotulo', 'Sentenca'])
gerarCSV("dsPriscilla", df)

Contabilizando divergencias entre os rotuladores

In [108]:
dados = zip(dsArthur, dsThiago, dsPriscilla)

divergenciaPriscilla = 0
divergenciaArthur = 0
divergenciaThiago = 0
divergenciaEntreOsTres = 0

for dadoArthur, dadoThiago, dadoPriscilla in dados:
    if dadoArthur[1] == dadoThiago[1] and dadoArthur[1] == dadoPriscilla[1]: ## Todos sendos iguais
        pass
    elif dadoArthur[1] == dadoThiago[1] and dadoArthur[1] != dadoPriscilla[1]: ## Priscilla divergindo
        divergenciaPriscilla += 1
    elif dadoArthur[1] == dadoPriscilla[1] and dadoArthur[1] != dadoThiago[1]: ## Thiago divergindo
        divergenciaThiago += 1
    elif dadoThiago[1] == dadoPriscilla[1] and dadoArthur[1] != dadoThiago[1]: ## Arthur divergindo
        divergenciaArthur += 1
    else:
        divergenciaEntreOsTres += 1

print(f'divergenciaPriscilla: {divergenciaPriscilla}\ndivergenciaArthur: {divergenciaArthur}\ndivergenciaThiago: {divergenciaThiago}\ndivergenciaEntreOsTres: {divergenciaEntreOsTres}')

divergenciaPriscilla: 3706
divergenciaArthur: 1846
divergenciaThiago: 3020
divergenciaEntreOsTres: 403


Resolvendo Divergencias

In [109]:
dadosFinais = []
textos = [i['data'] for i in lista_ArthurCompleta]
dados = zip(dsArthur, dsThiago, dsPriscilla)

# todas sentecas com discordancia entre os 3 rotuladores
for dadoArthur, dadoThiago, dadoPriscilla in dados:
    # print(len(dadoArthur))
    if dadoArthur[1] == dadoThiago[1] and dadoArthur[1] == dadoPriscilla[1]: ## Todos sendos iguais
        dadoArthur.append('[N]')
        dadosFinais.append(dadoArthur)
    elif dadoArthur[1] == dadoThiago[1] and dadoArthur[1] != dadoPriscilla[1]: ## Priscilla divergindo
        dadoArthur.append('[N]')
        dadosFinais.append(dadoArthur)
    elif dadoArthur[1] == dadoPriscilla[1] and dadoArthur[1] != dadoThiago[1]: ## Thiago divergindo
        dadoArthur.append('[N]')
        dadosFinais.append(dadoArthur)
    elif dadoThiago[1] == dadoPriscilla[1] and dadoArthur[1] != dadoThiago[1]: ## Arthur divergindo
        dadoThiago.append('[N]')
        dadosFinais.append(dadoThiago)
    else:
        dadoThiago.append('[S]')
        dadosFinais.append(dadoThiago)
        

In [110]:
df = pd.DataFrame(dadosFinais, columns = ['Palavras', 'Rotulo', 'Sentenca','Divergencia'])
gerarCSV("Divergencias", df)

Gerando Base final

In [54]:
df = pd.DataFrame(dadosFinais, columns = ['Palavras', 'Rotulo', 'Sentenca'])
gerarCSV("DS-BIO-IOB", df)

## Métricas

In [20]:
## palavras contém os tokens das sentenças que foram tokenizadas, dos arquivos de entradas.
palavras = [i[0] for i in dadosArthurFranco]

# printando diferenças
# for art, thi, pri in zip(palavras,palavrasThiago,palavrasPriscill):
#     if art != thi or art != pri or thi != pri:
#         print(f'Arthur: {art}\nThiago: {thi}\nPriscilla: {pri}\n')
    
df = pd.DataFrame({
     'palavrasArthur': dadosArthurFranco,
     'palavrasThiago': dadosThiagoSalles,
     'palavrasPriscill': dadosPriscillSilva
    })

gerarCSV("palavras", df)

## Pega as rotulações feitas por cada pessoa
rotulacoesArthurFranco = [i[1] for i in dadosArthurFranco]
rotulacoesPriscillSilva = [i[1] for i in dadosPriscillSilva]
rotulacoesThiagoSalles = [i[1] for i in dadosThiagoSalles]

df = pd.DataFrame({'palavras': palavras,
     'Arthur': rotulacoesArthurFranco,
     'Thiago': rotulacoesThiagoSalles,
     'Priscilla': rotulacoesPriscillSilva
    })
print(dadosArthurFranco[0])
gerarCSV("completo", df)

# Tirando palavras nao anotadas por ninguem 
dados = zip(palavras, rotulacoesArthurFranco, rotulacoesThiagoSalles, rotulacoesPriscillSilva)
semNone = [x for x in list(dados) if (x[1], x[2], x[3]) != ('o', 'o', 'o')]
palavraSNones, rotulacoesSNoneArthurFranco, rotulacoesSNoneThiagoSalles, rotulacoesSNonePriscillSilva = zip(*semNone)

df = pd.DataFrame({'palavras': palavraSNones,
     'Arthur': rotulacoesSNoneArthurFranco,
     'Thiago': rotulacoesSNoneThiagoSalles,
     'Priscilla': rotulacoesSNonePriscillSilva
    })

gerarCSV("semNone", df)

['id', 'o']


In [21]:
## Calcula a metrica cohen

dados = zip(rotulacoesArthurFranco, rotulacoesThiagoSalles)
semNone = [x for x in list(dados) if x != ('o', 'o')]
arthur, thiago = zip(*semNone)
print(cohen_kappa(arthur,thiago))

0.8748


In [51]:
## Calcula a metrica Fleiss desconsiderando None

categorias = ["NOME_BEBIDA", "GRADUACAO_ALCOOLICA", "EQUIPAMENTO_DESTILACAO",
            "TEMPO_ARMAZENAMENTO", "RECIPIENTE_ARMAZENAMENTO", "TIPO_MADEIRA",
            "CARACTERISTICA_SENSORIAL_COR", "CARACTERISTICA_SENSORIAL_AROMA", 
            "CARACTERISTICA_SENSORIAL_SABOR", "CARACTERISTICA_SENSORIAL_CONSISTÊNCIA", 
            "NOME_PESSOA", "NOME_LOCAL", "NOME_ORGANIZACAO", "TEMPO", "PRECO", "VOLUME", "CLASSIFICACAO_BEBIDA", "o"]

dados = zip(rotulacoesArthurFranco, rotulacoesThiagoSalles, rotulacoesPriscillSilva)
semNone = [x for x in list(dados) if x != ('o', 'o', 'o')]
arthur, thiago, priscilla = zip(*semNone)

M = np.zeros((len(arthur), len(categorias)))

nLines, nColumns = M.shape

for i in range(nLines):
    # if (arthur[i] != "o"):
        M[i, categorias.index(arthur[i])] += 1
    # if (thiago[i] != "o"):
        M[i, categorias.index(thiago[i])] += 1
    # if (priscilla[i] != "o"):
        M[i, categorias.index(priscilla[i])] += 1

# M.delete()

print(fleiss_kappa(M))

nan


/tmp/ipykernel_19922/3144744850.py:23: RuntimeWarning: invalid value encountered in true_divide
  P = (np.sum(M * M, axis=1) - n_annotators) / (n_annotators * (n_annotators - 1))


In [41]:
M[13, :]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [45]:
## Calcula a metrica Fleiss considerando None

categorias = ["NOME_BEBIDA", "GRADUACAO_ALCOOLICA", "EQUIPAMENTO_DESTILACAO",
            "TEMPO_ARMAZENAMENTO", "RECIPIENTE_ARMAZENAMENTO", "TIPO_MADEIRA",
            "CARACTERISTICA_SENSORIAL_COR", "CARACTERISTICA_SENSORIAL_AROMA", 
            "CARACTERISTICA_SENSORIAL_SABOR", "CARACTERISTICA_SENSORIAL_CONSISTÊNCIA", 
            "NOME_PESSOA", "NOME_LOCAL", "NOME_ORGANIZACAO", "TEMPO", "PRECO", "VOLUME", "CLASSIFICACAO_BEBIDA", "o"]

M_com_None = np.zeros((len(rotulacoesArthurFranco), len(categorias)))

nLines, nColumns = M_com_None.shape

for i in range(nLines):
    M_com_None[i, categorias.index(rotulacoesArthurFranco[i])] += 1
    M_com_None[i, categorias.index(rotulacoesThiagoSalles[i])] += 1
    M_com_None[i, categorias.index(rotulacoesPriscillSilva[i])] += 1


print(fleiss_kappa(M_com_None))

558057.0
558057.0
0.9203


In [ ]:
dados = zip(rotulacoesArthurFranco, rotulacoesThiagoSalles, rotulacoesPriscillSilva, palavras)
dadosFinais = [] 
for index, dado in enumerate(dados):
    if dado[0] == dado[1] and dado[0] == dado[2]: ## Todos sendos iguais
        dadosFinais.append(dado[0])
    elif dado[0] == dado[1] and dado[0] != dado[2]: ## Priscilla divergindo
        dadosFinais.append(dado[0])
    elif dado[0] == dado[2] and dado[0] != dado[1]: ## Thiago divergindo
        dadosFinais.append(dado[0])
    elif dado[1] == dado[2] and dado[1] != dado[0]: ## Arthur divergindo
        dadosFinais.append(dado[1])
    else:
        # """
        print("Houve divergência dos 3 membros na palavra: (" + dado[3] + ") no texto:\n" + 
            str(referencia[index][0]) + 
            "\nsegue as respectivas rotulacoes:\n" + 
            "Arthur: " + (str(dado[0]) if dado[0] != 'o' else "Nao rotulou") + "\n"
            "Thiago: " + (str(dado[1]) if dado[1] != 'o' else "Nao rotulou") + "\n"
            "Priscilla: " + (str(dado[2]) if dado[2] != 'o' else "Nao rotulou") + "\n"
             )
        resposta = int(input())
        print(f'resposta {resposta}')
        # """
        dadosFinais.append(dado[resposta])

In [ ]:
sentencas_originais = [x[0] for x in referencia]
index_originais = [x[1] for x in referencia]
posicao_inicial_palavra = [x[2][0] for x in referencia]
posicao_final_palavra = [x[2][1] for x in referencia]

df = pd.DataFrame({'palavras': palavras,
     'Rotulacoes': dadosFinais,
    'sentenca_original': sentencas_originais,
    'index_original': index_originais,
    'posicao_inicial_palavra': posicao_inicial_palavra,
    'posicao_final_palavra': posicao_final_palavra
    })

gerarCSV("Base Final", df)

In [18]:
display(df)

,palavras,Rotulacoes,sentenca_original,index_original,posicao_inicial_palavra,posicao_final_palavra
0,id,o,id: 0,0,0,2
1,:,o,id: 0,0,2,3
2,0,o,id: 0,0,4,5
3,NOME,o,NOME DA CACHAÇA: Cachaça Três Coronéis,1,0,4
4,DA,o,NOME DA CACHAÇA: Cachaça Três Coronéis,1,5,7
...,...,...,...,...,...,...
186014,Destilados,o,PREMIOS: Medalha de Prata no 17º concurso de V...,14286,54,64
186015,do,o,PREMIOS: Medalha de Prata no 17º concurso de V...,14286,65,67
186016,Brasil,NOME_LOCAL,PREMIOS: Medalha de Prata no 17º concurso de V...,14286,68,74
186017,2019,TEMPO,PREMIOS: Medalha de Prata no 17º concurso de V...,14286,75,79


## Modificando Inicio e Fim para cada token no dataset

In [5]:
# reading sentences
with open("../../DataSets/Dados_Utilizados_Para_RotulacaoManual.txt") as file:
    fixed_sentencas = [line.rstrip() for line in file]
print(len(fixed_sentencas))

13858


In [10]:
# reading database
finalDS = pd.read_csv("../../DataSets/Dataset_Padrao_IOB_Versao2.csv", usecols=['Palavras', 'Rotulo', 'Sentenca'])
len(finalDS)

186019

In [8]:
# token's position's in the setence
positions = []
for sent in fixed_sentencas:
    wordSpanList = list(TreebankWordTokenizer().span_tokenize(sent))
    for wordSpan in wordSpanList:
        # posicoes
        positions.append([wordSpan[0], wordSpan[1]])
len(positions)

186019

In [12]:
positions = pd.DataFrame(positions, columns = ['Inicio','Fim'])
start = positions['Inicio']
end = positions['Fim']

In [13]:
finalDS = finalDS.join(start)
finalDS = finalDS.join(end)
finalDS

,Palavras,Rotulo,Sentenca,Inicio,Fim
0,id,O,1,0,2
1,:,O,1,2,3
2,0,O,1,4,5
3,NOME,O,2,0,4
4,DA,O,2,5,7
...,...,...,...,...,...
186014,Destilados,O,13858,54,64
186015,do,O,13858,65,67
186016,Brasil,B-NOME_LOCAL,13858,68,74
186017,2019,B-TEMPO,13858,75,79


In [14]:
finalDS.to_csv("../../DataSets/Dataset_Padrao_IOB_Versao3.csv", encoding="utf-8")

## Adicionando Inicio e Fim para cada token no dataset

In [12]:
dados, referencia = tokenizeAn(lista_ArthurCompleta)
finalDS = pd.read_csv("./DataSet/Divergencias.csv", usecols=['Palavras', 'Rotulo', 'Sentenca'])


In [13]:
positions = [x[2] for x in referencia]
positions = pd.DataFrame(positions, columns = ['Inicio','Fim'])


In [15]:
start = positions['Inicio']
end = positions['Fim']

In [16]:
finalDS = finalDS.join(start)
finalDS = finalDS.join(end)
finalDS

,Palavras,Rotulo,Sentenca,Inicio
0,id,O,1,0
1,:,O,1,2
2,0,O,1,4
3,NOME,O,2,0
4,DA,O,2,5
...,...,...,...,...
186014,Destilados,O,14287,54
186015,do,O,14287,65
186016,Brasil,B-NOME_LOCAL,14287,68
186017,2019,B-TEMPO,14287,75


In [38]:
finalDS.to_csv("./DataSet/FinalDataSet.csv", encoding="utf-8")

## Transforma DataSet para formato Spacy

In [8]:
finalDS = pd.read_csv("./DataSet/FinalDataSet.csv", usecols=['Palavras', 'Rotulo', 'Sentenca', 'Inicio', 'Fim'])

# Pontos errados da Base

In [23]:
prevRot = 'O'
for index, row in finalDS.iterrows():
    if row['Rotulo'][0] == 'I' and prevRot == 'O':
        print(index)
        row['Rotulo'] = 'B' + row['Rotulo'][1:]
    prevRot = row['Rotulo'][0]
    # print(prevRot)


3319
7732
8191
25503
25785
55723
57586
77112
80014
80070
84265
87103
115590
121004
123126
126136
126778
130505
159364
162519
163087
163903
165758
171596
171606
172905
174275
182937
182978


## To SpaCy

In [5]:
# TODO - Ler textos.
with open("./DataSet/Dados_Utilizados_Para_RotulacaoManual.txt", "r") as file:
    textos = file.read().splitlines()

len(textos)


14287

In [18]:
sMarker = 0

s = textos[sMarker]
TRAIN_DATA = []
entities = []
for index, row in finalDS.iterrows():
    
    if(row['Sentenca']-1 != sMarker): # Nova sentença
        if entities:
            TRAIN_DATA.append((s, {"entities": [tuple(e) for e in entities]})) # Salva sentença anterior
        entities = [] # esvazia entidades
        sMarker = row['Sentenca']-1 # atualiza o marcador de sentença
        
        if (sMarker < len(textos)): # Limite de textos
            s = textos[sMarker]

    if(row['Rotulo'][0] == 'B'):
         entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
    if(row['Rotulo'][0] == 'I'):
        if (entities): 
            entities[-1][1] = row['Fim']
        else:
            # print(index)
            entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
    
    if (index == finalDS.index[-1]): # Ultimo elemento
        if entities:
            TRAIN_DATA.append((s, {"entities": [tuple(e) for e in entities]})) # Salva sentença anterior
        
        entities = [] 


len(TRAIN_DATA)
    
    



10969

In [17]:
# Usar pickle pra gerar .bin
# print(type(TRAIN_DATA[0][1]['entities']))
cont = 0
for data in TRAIN_DATA:
    if( not data[1]['entities']):
        cont = cont + 1

print(cont)

3318


In [22]:
print(TRAIN_DATA[2])

('DESCRIÇÃO DA CACHAÇA: Cachaça Três Coronéis 700ml', {'entities': [(30, 43, 'NOME_BEBIDA'), (44, 49, 'VOLUME')]})
